<a href="https://colab.research.google.com/github/justinballas/WorkAutomation/blob/main/BICFR_Snippetsv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is used as a repository of various blocks of code to be used in the Best In Class Fund Review (BICFR) workflow. This notebook is broken down into the following sections:

1. Importing the appropriate files

2. Data Cleaning

3. Carrier Subaccount Analysis and Rankings

4. Best in Class Fund Selection

5. Excel Reporting

In [1]:
!pip install xlsxwriter

     |████████████████████████████████| 153kB 29.3MB/s 


In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xlsxwriter
from glob import glob
%matplotlib inline

In [44]:
pd.set_option('display.max_rows', 100)

1. Importing the appropriate files

In [45]:
#Initiate a list of carriers. this abbreviation will be used to import each file based on the carrier name and will double as the alias for each carrier for use throughout the Python script. 
carriers_list = ['COIL','LCL', 'NWD', 'PL', 'PRT', 'MM']

#Using Glob to import files based on carrier name
files = [glob('{}*'.format(carrier)) for carrier in carriers_list]
#Glob returns a list, so when using list comprehentions it will return a list of lists, we must convert to a flat list.
files = [item for elem in files for item in elem]
#Reading in the CSVs, skipfooter removes the summary statistics from the Morningstar output
files = [pd.read_csv(file, skipfooter=13, engine='python') for file in files]
#Converts to a dictionary to easily call DataFrames for each carrier. Dictionary structure is used for looping through different amounts carriers to increase reusability of scripts.
carrier_dfs = dict(zip(carriers_list, files))

2. Data Cleaning

In [46]:
#removing "\n" artifact caused by text wrap.
for carrier, df in carrier_dfs.items():
  df.rename(columns=lambda x: x.replace('\n',''), inplace=True)

In [47]:
#remove funds that are closed to new investment
for carrier in carriers_list:
  temp_df = carrier_dfs[carrier]
  temp_df.drop(temp_df[temp_df['Closed to New Inv']=='Yes'].index, inplace=True)

In [48]:
#Selecting columns used in BICFR
cols_list = ['Name',
 'Morningstar Category',
 'Advisor',
 'Total Ret 1 Yr (Mo-End) Base Currency',
 'Total Ret Annlzd 3 Yr (Mo-End) Base Currency',
 'Total Ret Annlzd 5 Yr (Mo-End) Base Currency',
 'Total Ret Annlzd 10 Yr (Mo-End) Base Currency',
 'Morningstar Rating Overall',
 'Morningstar Rating 3 Yr',
 'Morningstar Rating 5 Yr',
 'Morningstar Rating 10 Yr',
 'Total Ret % Rank Cat 1 Yr (Mo-End)',
 'Total Ret % Rank Cat 3 Yr (Mo-End)',
 'Total Ret % Rank Cat 5 Yr (Mo-End)',
 'Total Ret % Rank Cat 10 Yr (Mo-End)',
 'Manager Tenure (Average)',
 'Sharpe Ratio 3 Yr (Mo-End) Risk Currency',
 'Prospectus Net Expense Ratio']

for carrier, df in carrier_dfs.items():
  carrier_dfs[carrier] = df.loc[:,cols_list]

In [49]:
#Remove prefixes in Morningstar Category
replace_list = ['US VL Sub ', 'US Insurance ', 'US Fund ']

for carrier in carriers_list:
  temp_df = carrier_dfs[carrier]
  for phrase in replace_list:
    temp_df['Morningstar Category'] = temp_df['Morningstar Category'].str.replace(phrase, "")

In [ ]:
#Remove prefixes in fund names
#create a dict frequency table for phgrases in front of -,ybe ones that show up for often are likely unneeded prefixes.

In [50]:
#Concat into single dataframe with carrier and fund name as a multi index
[df.insert(loc=0, column='Carrier', value=carrier) for carrier, df in carrier_dfs.items()]
df = pd.concat(carrier_dfs.values())
df.set_index(['Carrier', 'Name'], inplace=True)

3. Carrier Subaccount Rankings and Analysis.

In [15]:
#creating a pivot table to aggregate carrier fund data for rankings.
pivot_table = pd.pivot_table(df, index='Carrier', fill_value=np.nan)

In [16]:
#Seperating into tables for different sheets
man_tenure = pd.merge(pivot_table['Manager Tenure (Average)'], pivot_table['Manager Tenure (Average)'].rank(ascending=False), how='left', left_index=True, right_index=True,suffixes=('', ' Rank'))

sharpe_ratio = pd.merge(pivot_table['Sharpe Ratio 3 Yr (Mo-End) Risk Currency'], pivot_table['Sharpe Ratio 3 Yr (Mo-End) Risk Currency'].rank(ascending=False), how='left', left_index=True, right_index=True,suffixes=('', ' Rank'))

exp_ratio = pd.merge(pivot_table['Prospectus Net Expense Ratio'], pivot_table['Prospectus Net Expense Ratio'].rank(), how='left', left_index=True, right_index=True,suffixes=('', ' Rank'))

ms_ratings_cols = ['Morningstar Rating 10 Yr', 'Morningstar Rating 5 Yr', 'Morningstar Rating 3 Yr', 'Morningstar Rating Overall']
ms_ratings =  pivot_table[ms_ratings_cols]
for col in ms_ratings_cols:
  ms_ratings["{} Rank".format(col)] = ms_ratings[col].rank(ascending=False)

rank_cat_cols = ['Total Ret % Rank Cat 1 Yr (Mo-End)', 'Total Ret % Rank Cat 3 Yr (Mo-End)', 'Total Ret % Rank Cat 5 Yr (Mo-End)', 'Total Ret % Rank Cat 10 Yr (Mo-End)']
rank_cat = pivot_table[rank_cat_cols]
for col in rank_cat_cols:
  rank_cat["{} Rank".format(col)] = rank_cat[col].rank()

total_ret_cols = ['Total Ret 1 Yr (Mo-End) Base Currency', 'Total Ret Annlzd 3 Yr (Mo-End) Base Currency', 'Total Ret Annlzd 5 Yr (Mo-End) Base Currency', 'Total Ret Annlzd 10 Yr (Mo-End) Base Currency']
total_ret = pivot_table.loc[:, total_ret_cols]

for col in total_ret_cols:
  total_ret["{} Rank".format(col)] = total_ret[col].rank(ascending=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [17]:
total_ret

,Total Ret 1 Yr (Mo-End) Base Currency,Total Ret Annlzd 3 Yr (Mo-End) Base Currency,Total Ret Annlzd 5 Yr (Mo-End) Base Currency,Total Ret Annlzd 10 Yr (Mo-End) Base Currency,Total Ret 1 Yr (Mo-End) Base Currency Rank,Total Ret Annlzd 3 Yr (Mo-End) Base Currency Rank,Total Ret Annlzd 5 Yr (Mo-End) Base Currency Rank,Total Ret Annlzd 10 Yr (Mo-End) Base Currency Rank
Carrier,,,,,,,,
COIL,38.592250,11.724265,12.238955,9.021091,1.0,3.0,1.0,4.0
LCL,33.319308,11.198095,11.031210,8.399826,6.0,6.0,6.0,6.0
MM,35.515934,11.529121,11.496154,8.756207,4.0,5.0,5.0,5.0
NWD,34.737285,11.662945,11.737122,9.081333,5.0,4.0,4.0,3.0
PL,36.574107,11.991696,12.100642,9.231386,2.0,1.0,2.0,2.0
PRT,35.577426,11.857200,12.014592,9.282747,3.0,2.0,3.0,1.0


In [18]:
pivot_table

,Manager Tenure (Average),Morningstar Rating 10 Yr,Morningstar Rating 3 Yr,Morningstar Rating 5 Yr,Morningstar Rating Overall,Prospectus Net Expense Ratio,Sharpe Ratio 3 Yr (Mo-End) Risk Currency,Total Ret % Rank Cat 1 Yr (Mo-End),Total Ret % Rank Cat 10 Yr (Mo-End),Total Ret % Rank Cat 3 Yr (Mo-End),Total Ret % Rank Cat 5 Yr (Mo-End),Total Ret 1 Yr (Mo-End) Base Currency,Total Ret Annlzd 10 Yr (Mo-End) Base Currency,Total Ret Annlzd 3 Yr (Mo-End) Base Currency,Total Ret Annlzd 5 Yr (Mo-End) Base Currency
Carrier,,,,,,,,,,,,,,,
COIL,7.376500,3.444444,3.388060,3.469697,3.492537,1.007000,0.622353,46.375000,44.000000,45.764706,41.835821,38.592250,9.021091,11.724265,12.238955
LCL,6.745328,3.070175,3.208000,3.178862,3.200000,0.682993,0.704841,46.162791,47.429825,44.192000,45.292683,33.319308,8.399826,11.198095,11.031210
MM,7.003981,3.883721,3.593407,3.626374,3.857143,0.813241,0.710659,39.637363,31.494253,38.593407,37.395604,35.515934,8.756207,11.529121,11.496154
NWD,7.401688,4.061538,3.703448,3.775362,4.020690,0.715649,0.710548,41.060000,28.681481,37.363014,35.381295,34.737285,9.081333,11.662945,11.737122
PL,7.483661,3.442105,3.405405,3.435185,3.504505,0.649911,0.695000,45.750000,70.000000,55.750000,49.000000,36.574107,9.231386,11.991696,12.100642
PRT,7.199604,3.977011,3.326531,3.541667,3.755102,0.816931,0.693700,50.485149,29.288889,46.600000,41.836735,35.577426,9.282747,11.857200,12.014592


In [19]:
def freq_table(data, column):
  table = {}
  for item in data[column].unique():
    freq = len(data[data[column]==item])
    table[item] = freq
  table = pd.DataFrame.from_dict(table, orient='index', columns=['Frequency']).sort_values(by='Frequency', ascending=False)
  return table
  

In [ ]:
# freq_table(df, 'Morningstar Category')

3. Best In Class Fund Selection

In [54]:
df['Sum Rating'] = df['Morningstar Rating 3 Yr'] + df['Morningstar Rating 5 Yr'] + df['Morningstar Rating 10 Yr'] + df['Morningstar Rating Overall']
df['Sum Ranking'] = df['Total Ret % Rank Cat 1 Yr (Mo-End)'] + df['Total Ret % Rank Cat 3 Yr (Mo-End)'] + df['Total Ret % Rank Cat 5 Yr (Mo-End)'] + df['Total Ret % Rank Cat 10 Yr (Mo-End)']
df['Sum Ranking'] = df['Sum Ranking'].apply(lambda x: 400-x)

test = df.loc[('PRT', slice(None))]
test = test[test['Morningstar Category'] == 'Large Blend']  

In [21]:
if len(test[test['Sum Rating'] == test['Sum Rating'].max()]['Sum Rating']) == 1:
  test['BICF'] = 'Yes'
elif len(test[test['Sum Ranking'] == test['Sum Ranking'].max()]['Sum Ranking']) == 1:
  test['BICF'] = 'Yes'
else:
  pass

In [ ]:
# def get_uniques(df_list, column, keep=str):
#   '''
#   get all unique values from a single column in multiple DataFrames
#   '''

#   vals = [list(df[column]) for df in df_list]
#   vals = set([item for elem in vals for item in elem])
#   vals = [e for e in vals if isinstance(e, keep)]
#   return list(vals)

# asset_classes = get_uniques(carrier_dfs.values(), 'Morningstar Category')

In [82]:
asset_classes = list(df['Morningstar Category'].dropna().unique())

In [91]:
asset_classes.sort()

In [92]:
asset_classes

['Allocation--15% to 30% Equity',
 'Allocation--30% to 50% Equity',
 'Allocation--50% to 70% Equity',
 'Allocation--70% to 85% Equity',
 'Allocation--85%+ Equity',
 'Bank Loan',
 'Commodities Broad Basket',
 'Convertibles',
 'Diversified Emerging Mkts',
 'Emerging Markets Bond',
 'Equity Energy',
 'Event Driven',
 'Financial',
 'Foreign Large Blend',
 'Foreign Large Growth',
 'Foreign Large Value',
 'Foreign Small/Mid Blend',
 'Fund World Large-Stock Growth',
 'Global Real Estate',
 'Health',
 'High Yield Bond',
 'Inflation-Protected Bond',
 'Intermediate Core Bond',
 'Intermediate Core-Plus Bond',
 'Intermediate Government',
 'Large Blend',
 'Large Growth',
 'Large Value',
 'Long Government',
 'Long-Short Equity',
 'Mid-Cap Blend',
 'Mid-Cap Growth',
 'Mid-Cap Value',
 'Money Market - Taxable',
 'Multisector Bond',
 'Multistrategy',
 'Natural Resources',
 'Nontraditional Bond',
 'Prime Money Market',
 'Real Estate',
 'Short-Term Bond',
 'Small Blend',
 'Small Growth',
 'Small Value',


In [ ]:
#Selects best in class funds based on the following order:
#   Highest sum of MS rating, if multiple funds have the same highest MS rating sum,
#   MS sum ranking is used, if MS rating is (closeness factor %) close to the highest
#   MS ranking sum, lowest expense ratio fund is used.

closeness_factor = .2
BICF_list = []
for asset_class in asset_classes:
  temp_df = df[df['Morningstar Category'] == asset_class]
  aclass_max_rating = temp_df['Sum Rating'].max()
  max_df = temp_df[temp_df['Sum Rating'] == aclass_max_rating]
  if len(max_df) == 1:
    print(max_df)
    BICF_list.append(max_df)
  elif len(max_df) > 1:
    aclass_max_ranking = max_df['Sum Ranking'].max()*(1-closeness_factor)
    filt = df['Sum Ranking'] > aclass_max_ranking
    max_df = max_df[filt]
  elif len(max_df) == 1:
    BICF_list.append(max_df)
  elif len(max_df) > 1:
    min_exp_ratio = max_df['Prospectus Net Expense Ratio'].min()
    max_df = max_df[max_df['Prospectus Net Expense Ratio'] == min_exp_ratio]
    BICF_list.append(max_df)
  else:
    pass

In [123]:
filt = df['Morningstar Category']=='Large Growth'
temp_df = df[filt].loc[('PRT', slice(None))]

In [126]:
pd.concat(BICF_list)

,Morningstar Category,Advisor,Total Ret 1 Yr (Mo-End) Base Currency,Total Ret Annlzd 3 Yr (Mo-End) Base Currency,Total Ret Annlzd 5 Yr (Mo-End) Base Currency,Total Ret Annlzd 10 Yr (Mo-End) Base Currency,Morningstar Rating Overall,Morningstar Rating 3 Yr,Morningstar Rating 5 Yr,Morningstar Rating 10 Yr,Total Ret % Rank Cat 1 Yr (Mo-End),Total Ret % Rank Cat 3 Yr (Mo-End),Total Ret % Rank Cat 5 Yr (Mo-End),Total Ret % Rank Cat 10 Yr (Mo-End),Manager Tenure (Average),Sharpe Ratio 3 Yr (Mo-End) Risk Currency,Prospectus Net Expense Ratio,Sum Rating,Sum Ranking
Name,,,,,,,,,,,,,,,,,,,
Pro Ex Br-American Funds IS® Growth 2,Large Growth,Capital Research and Management Company,54.76,25.9,24.5,16.98,5.0,5.0,5.0,4.0,5.0,5.0,6.0,19.0,4.31,1.19,0.61,19.0,365.0


In [128]:
temp_df[temp_df['Sum Rating']==19]

,Morningstar Category,Advisor,Total Ret 1 Yr (Mo-End) Base Currency,Total Ret Annlzd 3 Yr (Mo-End) Base Currency,Total Ret Annlzd 5 Yr (Mo-End) Base Currency,Total Ret Annlzd 10 Yr (Mo-End) Base Currency,Morningstar Rating Overall,Morningstar Rating 3 Yr,Morningstar Rating 5 Yr,Morningstar Rating 10 Yr,Total Ret % Rank Cat 1 Yr (Mo-End),Total Ret % Rank Cat 3 Yr (Mo-End),Total Ret % Rank Cat 5 Yr (Mo-End),Total Ret % Rank Cat 10 Yr (Mo-End),Manager Tenure (Average),Sharpe Ratio 3 Yr (Mo-End) Risk Currency,Prospectus Net Expense Ratio,Sum Rating,Sum Ranking
Name,,,,,,,,,,,,,,,,,,,
Pro Ex Br-American Funds IS® Growth 2,Large Growth,Capital Research and Management Company,54.76,25.90,24.50,16.98,5.0,5.0,5.0,4.0,5.0,5.0,6.0,19.0,4.31,1.19,0.61,19.0,365.0
Pro Ex Br-Janus Henderson VIT Forty Instl,Large Growth,Janus Capital Management LLC,43.94,24.71,22.94,17.65,5.0,5.0,4.0,5.0,24.0,15.0,25.0,7.0,6.71,1.20,0.76,19.0,329.0
